In [1]:
import pandas as pd
import numpy as np
import cv2
import os
import re
from scipy.optimize import linear_sum_assignment

In [2]:
bodiesDF = pd.read_pickle('synchronization/synchronize')
facesDF = pd.read_pickle('faces')
uniqueNames = np.load('uniqueName.npy')

In [3]:
for n in uniqueNames.tolist():
    bodiesDF[n] = np.nan

In [4]:
bodiesDF

traSeq  trackId  trackX  trackY  trackW  trackH    height  distance  \
seq  id                                                                        
784  12     784       12   238.0   278.0   336.6   396.0  1.597804  2.993590   
     14     784       14    81.6   400.0    91.8   108.0  0.803921  5.991756   
785  12     785       12   217.6   276.0   343.4   406.0  1.598425  2.921760   
     14     785       14    81.6   400.0    91.8   108.0  0.803921  5.991756   
786  12     786       12   193.8   276.0   357.0   420.0  1.599651  2.825866   
     14     786       14    81.6   400.0    91.8   108.0  0.803921  5.991756   
787  12     787       12   170.0   276.0   367.2   432.0  1.598773  2.788292   
     14     787       14    81.6   400.0    91.8   108.0  0.803921  5.991756   
788  12     788       12   163.2   278.0   370.6   436.0  1.598104  2.776649   
     14     788       14    81.6   400.0    91.8   108.0  0.803921  5.991756   
789  12     789       12   156.4   280.0   374.0   440.0  1.597755  2.751087   
     14     789       14    81.6   400.0    91.8   108.0  0.803921  5.991756   
790  12     790       12   146.2   282.0   377.4   446.0  1.597284  2.733374   
     14     790       14    81.6   400.0    91.8   108.0  0.803921  5.991756   
791  12     791       12   142.8   284.0   380.8   450.0  1.597968  2.722706   
     14     791       14    81.6   400.0    91.8   108.0  0.803921  5.991756   
792  12     792       12   122.4   284.0   394.4   464.0  1.596457  2.655667   
     14     792       14    81.6   400.0    91.8   108.0  0.803921  5.991756   
793  12     793       12   112.2   284.0   404.6   478.0  1.594237  2.590717   
     14     793       14    81.6   400.0    91.8   108.0  0.803921  5.991756   
794  12     794       12   115.6   286.0   414.8   488.0  1.592291  2.487561   
     14     794       14    81.6   400.0    91.8   108.0  0.803921  5.991756   
795  12     795       12   136.0   288.0   418.2   494.0  1.590862  2.418992   
     14     795       14    81.6   400.0    91.8   108.0  0.803921  5.991756   
796  12     796       12   146.2   288.0   421.6   498.0  1.592582  2.341361   
     14     796       14    81.6   400.0    91.8   108.0  0.803921  5.991756   
797  12     797       12   159.8   290.0   428.4   506.0  1.594597  2.298524   
     14     797       14    81.6   400.0    91.8   108.0  0.803921  5.991756   
798  12     798       12   180.2   290.0   442.0   520.0  1.595444  2.283207   
     14     798       14    81.6   400.0    91.8   108.0  0.803921  5.991756   
...         ...      ...     ...     ...     ...     ...       ...       ...   
1137 17    1137       17  1186.6   616.0   119.0   140.0  0.494020  2.573133   
1138 12    1138       12   547.4   262.0   397.8   470.0  1.583883  2.319799   
     15    1138       15   673.2   318.0   275.4   326.0  1.538408  3.255186   
     17    1138       17  1186.6   616.0   119.0   140.0  0.494020  2.573133   
1141 12    1141       12   547.4   262.0   397.8   470.0  1.583883  2.319799   
     15    1141       15   615.4   322.0   261.8   310.0  1.531690  3.379526   
1142 12    1142       12   710.6   264.0   404.6   478.0  1.584980  2.249599   
     15    1142       15   588.2   320.0   261.8   310.0  1.529408  3.340304   
1143 12    1143       12   724.2   270.0   401.2   474.0  1.586450  2.234758   
     15    1143       15   574.6   318.0   261.8   310.0  1.524919  3.354659   
1144 12    1144       12   771.8   272.0   401.2   474.0  1.588640  2.246878   
     15    1144       15   557.6   316.0   261.8   310.0  1.524032  3.373454   
1145 12    1145       12   785.4   274.0   401.2   472.0  1.591561  2.254036   
     15    1145       15   550.8   316.0   261.8   310.0  1.523025  3.387116   
1146 12    1146       12   822.8   276.0   401.2   472.0  1.593780  2.266992   
     15    1146       15   530.4   314.0   265.2   312.0  1.525077  3.374292   
1147 12    1147       12   884.0   280.0   397.8   468.0  1.595187  2.330174   
     15  

In [13]:
for seq in bodiesDF.rgbSeq.unique():
    #print seq
    bigDist = []
    bodies = bodiesDF[bodiesDF.rgbSeq == seq].reset_index().drop_duplicates(subset='id', keep='first')
    faces = facesDF[facesDF.rgbSeq == seq]
    
    for fi, f in faces.iterrows():
        faceCentroid = (int(f.faceX), int(f.faceY))
        distance = []
        if len(bodies) > 0:
            for bi, b in bodies.iterrows():
                upLeft = tuple((int(b.trackX), int(b.trackY)))
                downRight = tuple((int(b.trackX) + int(b.trackW), int(b.trackY) + int(b.trackH)))
                bodyCentroid = tuple((np.asarray(upLeft) + np.asarray(downRight)) / 2)
                
                inside = (bodyCentroid[0] - faceCentroid[0])**2 + (bodyCentroid[1] - faceCentroid[1])**2 < (b.trackH.astype(int)/2)**2
                up = bodyCentroid[1] > faceCentroid[1]
                if inside and up:
                    dist = ((bodyCentroid[0] - faceCentroid[0])**2 + (bodyCentroid[1] - faceCentroid[1])**2)**(0.5)
                    distance.append(dist)
                else:
                    distance.append(2237)
        bigDist.append(distance)

    if len(bigDist) > 0:
        row_ind, col_ind = linear_sum_assignment(np.asarray(bigDist).T)
        for i in range(len(row_ind)):
            b = bodies.iloc[row_ind[i]]
            #print int(b.seq)
            #print int(b.id)
            for n in uniqueNames.tolist():
                #print bodiesDF[[int(b.seq), int(b.id)]]
                bodiesDF.loc.__setitem__(((int(b.seq), int(b.id)),n), faces.iloc[col_ind[i]][n])

In [14]:
bodiesDF

traSeq  trackId  trackX  trackY  trackW  trackH    height  distance  \
seq  id                                                                        
784  12     784       12   238.0   278.0   336.6   396.0  1.597804  2.993590   
     14     784       14    81.6   400.0    91.8   108.0  0.803921  5.991756   
785  12     785       12   217.6   276.0   343.4   406.0  1.598425  2.921760   
     14     785       14    81.6   400.0    91.8   108.0  0.803921  5.991756   
786  12     786       12   193.8   276.0   357.0   420.0  1.599651  2.825866   
     14     786       14    81.6   400.0    91.8   108.0  0.803921  5.991756   
787  12     787       12   170.0   276.0   367.2   432.0  1.598773  2.788292   
     14     787       14    81.6   400.0    91.8   108.0  0.803921  5.991756   
788  12     788       12   163.2   278.0   370.6   436.0  1.598104  2.776649   
     14     788       14    81.6   400.0    91.8   108.0  0.803921  5.991756   
789  12     789       12   156.4   280.0   374.0   440.0  1.597755  2.751087   
     14     789       14    81.6   400.0    91.8   108.0  0.803921  5.991756   
790  12     790       12   146.2   282.0   377.4   446.0  1.597284  2.733374   
     14     790       14    81.6   400.0    91.8   108.0  0.803921  5.991756   
791  12     791       12   142.8   284.0   380.8   450.0  1.597968  2.722706   
     14     791       14    81.6   400.0    91.8   108.0  0.803921  5.991756   
792  12     792       12   122.4   284.0   394.4   464.0  1.596457  2.655667   
     14     792       14    81.6   400.0    91.8   108.0  0.803921  5.991756   
793  12     793       12   112.2   284.0   404.6   478.0  1.594237  2.590717   
     14     793       14    81.6   400.0    91.8   108.0  0.803921  5.991756   
794  12     794       12   115.6   286.0   414.8   488.0  1.592291  2.487561   
     14     794       14    81.6   400.0    91.8   108.0  0.803921  5.991756   
795  12     795       12   136.0   288.0   418.2   494.0  1.590862  2.418992   
     14     795       14    81.6   400.0    91.8   108.0  0.803921  5.991756   
796  12     796       12   146.2   288.0   421.6   498.0  1.592582  2.341361   
     14     796       14    81.6   400.0    91.8   108.0  0.803921  5.991756   
797  12     797       12   159.8   290.0   428.4   506.0  1.594597  2.298524   
     14     797       14    81.6   400.0    91.8   108.0  0.803921  5.991756   
798  12     798       12   180.2   290.0   442.0   520.0  1.595444  2.283207   
     14     798       14    81.6   400.0    91.8   108.0  0.803921  5.991756   
...         ...      ...     ...     ...     ...     ...       ...       ...   
1137 17    1137       17  1186.6   616.0   119.0   140.0  0.494020  2.573133   
1138 12    1138       12   547.4   262.0   397.8   470.0  1.583883  2.319799   
     15    1138       15   673.2   318.0   275.4   326.0  1.538408  3.255186   
     17    1138       17  1186.6   616.0   119.0   140.0  0.494020  2.573133   
1141 12    1141       12   547.4   262.0   397.8   470.0  1.583883  2.319799   
     15    1141       15   615.4   322.0   261.8   310.0  1.531690  3.379526   
1142 12    1142       12   710.6   264.0   404.6   478.0  1.584980  2.249599   
     15    1142       15   588.2   320.0   261.8   310.0  1.529408  3.340304   
1143 12    1143       12   724.2   270.0   401.2   474.0  1.586450  2.234758   
     15    1143       15   574.6   318.0   261.8   310.0  1.524919  3.354659   
1144 12    1144       12   771.8   272.0   401.2   474.0  1.588640  2.246878   
     15    1144       15   557.6   316.0   261.8   310.0  1.524032  3.373454   
1145 12    1145       12   785.4   274.0   401.2   472.0  1.591561  2.254036   
     15    1145       15   550.8   316.0   261.8   310.0  1.523025  3.387116   
1146 12    1146       12   822.8   276.0   401.2   472.0  1.593780  2.266992   
     15    1146       15   530.4   314.0   265.2   312.0  1.525077  3.374292   
1147 12    1147       12   884.0   280.0   397.8   468.0  1.595187  2.330174   
     15  

In [15]:
bodiesDF.to_pickle('bodyFace')

In [114]:
'''seqM = []
idM = []
indexM = []

for seq in bodiesDF.rgbSeq.unique():
    #seq = 14
    bodies = bodiesDF[bodiesDF.rgbSeq == seq].reset_index().drop_duplicates(subset='id', keep='first')
    faces = facesDF[facesDF.rgbSeq == seq]
    
    img = cv2.imread("/home/sabrine/notebook/modified2/" + str(seq) + ".jpg")
    
    for index, face in faces.iterrows():
        faceCentroid = (int(face.faceX), int(face.faceY))
        distance = []
        for i, b in bodies.iterrows():
            upLeft = tuple((b.trackX, b.trackY))
            downRight = tuple((b.trackX + b.trackW, b.trackY + b.trackH))
            bodyCentroid = tuple((np.asarray(upLeft) + np.asarray(downRight)) / 2)
            
            
            inside = (bodyCentroid[0] - faceCentroid[0])**2 + (bodyCentroid[1] - faceCentroid[1])**2 < (b.trackH.astype(int)/2)**2
            up = bodyCentroid[1] > faceCentroid[1]
            if inside and up:
                dist = ((bodyCentroid[0] - faceCentroid[0])**2 + (bodyCentroid[1] - faceCentroid[1])**2)**(0.5)
                distance.append(dist)
                
                color = (255, 255, 64)
                thickness = 1
                lineType = 8 # will discorver what it means
                radius = b.trackH.astype(int)/2
                img = cv2.circle(img, faceCentroid, int(radius), color, thickness, lineType)
            else:
                distance.append(np.inf)
                
        argMin = np.argmin(np.asarray(distance))
        seqMin = int(bodies.iloc[argMin].seq)
        idMin = int(bodies.iloc[argMin].id)
        seqM.append(seqMin)
        idM.append(idMin)
        indexM.append(index)
        
                
    cv2.imwrite("/home/sabrine/notebook/modified/" + str(seq) + ".jpg", img)'''

'seqM = []\nidM = []\nindexM = []\n\nfor seq in bodiesDF.rgbSeq.unique():\n    #seq = 14\n    bodies = bodiesDF[bodiesDF.rgbSeq == seq].reset_index().drop_duplicates(subset=\'id\', keep=\'first\')\n    faces = facesDF[facesDF.rgbSeq == seq]\n    \n    img = cv2.imread("/home/sabrine/notebook/modified2/" + str(seq) + ".jpg")\n    \n    for index, face in faces.iterrows():\n        faceCentroid = (int(face.faceX), int(face.faceY))\n        distance = []\n        for i, b in bodies.iterrows():\n            upLeft = tuple((b.trackX, b.trackY))\n            downRight = tuple((b.trackX + b.trackW, b.trackY + b.trackH))\n            bodyCentroid = tuple((np.asarray(upLeft) + np.asarray(downRight)) / 2)\n            \n            \n            inside = (bodyCentroid[0] - faceCentroid[0])**2 + (bodyCentroid[1] - faceCentroid[1])**2 < (b.trackH.astype(int)/2)**2\n            up = bodyCentroid[1] > faceCentroid[1]\n            if inside and up:\n                dist = ((bodyCentroid[0] - faceCent

In [115]:
'''for seq in bodiesDF.rgbSeq.unique():
    #seq = 14
    bodies = bodiesDF[bodiesDF.rgbSeq == seq].reset_index().drop_duplicates(subset='id', keep='first')
    faces = facesDF[facesDF.rgbSeq == seq]
    
    print bodies
    print '#####'
    
    img = cv2.imread("/home/sabrine/notebook/modified2/" + str(seq) + ".jpg")
    
    for index, face in faces.iterrows():
        faceCentroid = (int(face.faceX), int(face.faceY))
        
        upLeft = np.asarray([bodies.trackX, bodies.trackY])
        downRight = np.asarray([bodies.trackX + bodies.trackW, bodies.trackY + bodies.trackH])
        print upLeft
        print downRight
        print type(downRight)
        
        bodyCentroid = (upLeft[O].add(downRight)).divide(2)
            
        inside = (bodyCentroid[0] - faceCentroid[0])**2 + (bodyCentroid[1] - faceCentroid[1])**2 < (b.trackH.astype(int)/2)**2
        up = bodyCentroid[1] > faceCentroid[1]
        if inside and up:
            color = (255, 255, 64)
            thickness = 1
            radius = b.trackH.astype(int)/2
            img = cv2.circle(img, faceCentroid, int(radius), color, thickness, lineType)'''

'for seq in bodiesDF.rgbSeq.unique():\n    #seq = 14\n    bodies = bodiesDF[bodiesDF.rgbSeq == seq].reset_index().drop_duplicates(subset=\'id\', keep=\'first\')\n    faces = facesDF[facesDF.rgbSeq == seq]\n    \n    print bodies\n    print \'#####\'\n    \n    img = cv2.imread("/home/sabrine/notebook/modified2/" + str(seq) + ".jpg")\n    \n    for index, face in faces.iterrows():\n        faceCentroid = (int(face.faceX), int(face.faceY))\n        \n        upLeft = np.asarray([bodies.trackX, bodies.trackY])\n        downRight = np.asarray([bodies.trackX + bodies.trackW, bodies.trackY + bodies.trackH])\n        print upLeft\n        print downRight\n        print type(downRight)\n        \n        bodyCentroid = (upLeft[O].add(downRight)).divide(2)\n            \n        inside = (bodyCentroid[0] - faceCentroid[0])**2 + (bodyCentroid[1] - faceCentroid[1])**2 < (b.trackH.astype(int)/2)**2\n        up = bodyCentroid[1] > faceCentroid[1]\n        if inside and up:\n            color = (255

In [116]:
'''for seq in bodiesDF.rgbSeq.unique():
    bodies = bodiesDF[bodiesDF.rgbSeq == seq]
    faces = facesDF[facesDF.rgbSeq == seq]
    
    #print 'bodies'
    #print bodies
    #print '####'
    #print 'faces'
    #print faces
    for i, f in faces.iterrows():
        if len(bodies) > 0 :        
            
            candidates = bodies[
                [((((bodies.trackX.astype(int) + bodies.trackW.astype(int)) / 2 ) - f.faceX)**2 + 
                (((bodies.trackY.astype(int) + bodies.trackH.astype(int)) / 2 ) - f.faceY)**2) < 
                 (bodies.trackH.astype(int)/2)**2] # 250 arbitrary picked 
                and 
                ((bodies.trackY.astype(int) + bodies.trackH.astype(int)) / 2 ) < f.faceY
            ]
            if not candidates.empty:
                print '******'
                print f
                print candidates
                print '******''''

SyntaxError: EOL while scanning string literal (<ipython-input-116-eb0dcd4023a7>, line 24)